In [4]:
type State = Int
type Delta g = [(State, Maybe Char, Maybe g, [g], State)]

delta :: Delta g -> (State)

-- (max st, transitions, init stack sym, init st, final st)
type PDA g = (State, Delta g, g, State, [State])